In [2]:
from implementations import *
from data_cleanup import *
from functools import partial
np.warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

In [54]:
DATA_TRAIN_PATH = 'data/train.csv' 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

Test data cleaning functions

In [55]:
tX = undefined_to_nans(tX)
tX = nans_to_means(tX)
tX = remove_zscore_outliers(tX)
tX, y = remove_nans(tX, y)
y[y == -1] = 0
print(tX.shape, y.shape)

(210385, 30) (210385,)


In [5]:
initial_w = np.zeros(30)
gamma = 0.01
max_iters = 10
batch_size = 10000
ratio = 0.8

Test logistic regression using Newton method

In [27]:
w , loss = logistic_regression_newton_method(y, tX, initial_w, max_iters,batch_size, ratio, gamma)

epoch: 0, train_loss: 78130.99788099548, test_loss: 20120.34454764845, test accuracy: 0.7398
epoch: 1, train_loss: 70202.42585771729, test_loss: 18234.339692503512, test accuracy: 0.73994
epoch: 2, train_loss: 64308.72898050649, test_loss: 16820.910053378484, test accuracy: 0.7402
epoch: 3, train_loss: 59853.98692328149, test_loss: 15739.335999859992, test accuracy: 0.74048
epoch: 4, train_loss: 56399.15235379851, test_loss: 14900.524733654807, test accuracy: 0.7404999999999999
epoch: 5, train_loss: 53715.799202397204, test_loss: 14244.625281790366, test accuracy: 0.7408600000000001
epoch: 6, train_loss: 51617.111259586876, test_loss: 13727.123038232334, test accuracy: 0.74114
epoch: 7, train_loss: 49944.90255395859, test_loss: 13318.37084489838, test accuracy: 0.74144
epoch: 8, train_loss: 48614.55956960761, test_loss: 12994.930274724416, test accuracy: 0.74124
epoch: 9, train_loss: 47567.90546295885, test_loss: 12737.76382052801, test accuracy: 0.74146


In [4]:
dummy_x = np.random.rand(100,3)
dummy_y = np.random.randint(2, size=100)
dummy_w = np.random.rand(3)

Test cross validation

In [6]:
model = partial(logistic_regression_newton_method, initial_w = dummy_w, max_iters = 1, batch_size = 3, ratio = 1, gamma = gamma)
loss = compute_cross_entropy_loss
cross_validation_score(model, dummy_x , dummy_y, loss, cv=10, seed=1)


(array([44.24772374, 38.23948318, 63.9058497 , 31.15436397, 60.6757757 ,
        53.75348344,         nan, 59.44028758, 63.53367256, 56.83503098]),
 array([3.50869022, 5.44816691, 9.19948827, 2.71733991, 4.85471977,
        9.42375412,        nan, 9.49667408, 6.88554625, 5.8570778 ]))

In [11]:
cross_validation_score(least_squares, dummy_x , dummy_y, compute_mse, cv=10, seed=1)

(array([0.28125694, 0.28942513, 0.29592614, 0.28621503, 0.28843707,
        0.30239561, 0.29338572, 0.29206854, 0.29401024, 0.29305165]),
 array([0.19877304, 0.16276634, 0.13168506, 0.17640766, 0.17777163,
        0.10731871, 0.14318173, 0.15490656, 0.14091203, 0.15884048]))

testing least square and ridge

In [9]:
ridge_regression(dummy_y, dummy_x, 0.5)
least_squares(dummy_y, dummy_x)

(array([0.28911307, 0.28053288, 0.22584739]), 0.29255096686754833)

Create submission

In [130]:
OUTPUT_PATH = 'data/test.csv' 
_, tX_test, ids_test = load_csv_data(OUTPUT_PATH)
y_test = predict_logistic_labels(w, tX_test)
y_test = y_test * 2 - 1

create_csv_submission(ids_test, y_test, 'bkm.csv')
